<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/WienerPathTensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf

from quant_analytics_flow.analytics.norminv import norminv
from quant_analytics_flow.calculators.univariatebrownianbridge import UnivariateBrownianBridge


In [16]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  2#@param {type:"integer"}
number =  7#@param {type:"integer"}

brownian = UnivariateBrownianBridge(dim)

x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
x = tf.transpose(x)
y = tf.reshape(x, shape=(dim,1,number))
z = norminv(y)

print(z)
print(brownian.path(z, False))
print(brownian.path(z, True))

tf.Tensor(
[[[ 0.          0.67448975 -0.67448975 -0.31863936  1.15034938
    0.31863936 -1.15034938]]

 [[ 0.         -0.67448975  0.67448975 -0.31863936  1.15034938
   -1.15034938  0.31863936]]], shape=(2, 1, 7), dtype=float64)
tf.Tensor(
[[[ 0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
    0.00000000e+00  0.00000000e+00 -0.00000000e+00]]

 [[ 0.00000000e+00 -2.44760480e-17  2.44760480e-17 -4.50624110e-01
    1.62683970e+00 -5.88107793e-01 -5.88107793e-01]]

 [[ 0.00000000e+00  9.53872552e-01 -9.53872552e-01 -4.50624110e-01
    1.62683970e+00  4.50624110e-01 -1.62683970e+00]]], shape=(3, 1, 7), dtype=float64)
tf.Tensor(
[[[ 0.00000000e+00 -2.44760480e-17  2.44760480e-17 -4.50624110e-01
    1.62683970e+00 -5.88107793e-01 -5.88107793e-01]]

 [[ 0.00000000e+00  9.53872552e-01 -9.53872552e-01  0.00000000e+00
    0.00000000e+00  1.03873190e+00 -1.03873190e+00]]

 [[ 0.00000000e+00  9.53872552e-01 -9.53872552e-01 -4.50624110e-01
    1.62683970e+00  4.50624110e-01 -1.62683

In [18]:
import time

In [23]:
device_name = tf.test.gpu_device_name()

''

In [80]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  256#@param {type:"integer"}
number =  200000#@param {type:"integer"}

time_start = time.time()
with tf.device("/cpu:0"):
    brownian = UnivariateBrownianBridge(dim)
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    x = tf.transpose(x)
    y = tf.reshape(x, shape=(dim,1,number))
    z = norminv(y)
    w = brownian.path(z, True)
time_end = time.time()
time_price_cpu = time_end - time_start
print("Runtime on CPU: ", time_price_cpu)

Runtime on CPU:  1.2129936218261719


In [81]:
from quant_analytics_flow.calculators.multivariatebrownianbridge import MultivariateBrownianBridge

rho = tf.Variable(0.5, dtype=tf.float64, name='rho')

times =  128#@param {type:"integer"}
number =  200000#@param {type:"integer"}

time_start = time.time()

with tf.GradientTape() as tape:
    with tf.device("/cpu:0"):
        fm = [[1.,rho],[rho,1.]]
        fwdCov = tf.TensorArray(dtype=tf.float64, size = times)
        fwdCov = fwdCov.write(0,fm)
        fwdCov = fwdCov.write(1,fm)
        fwdCov = fwdCov.write(2,fm)        

        multivariate_bridge = MultivariateBrownianBridge(fwdCov.stack())
        w = multivariate_bridge.path(number)

time_end = time.time()
time_price_cpu = time_end - time_start
print("Runtime on CPU: ", time_price_cpu)        

Runtime on CPU:  1.7069647312164307


In [72]:
time_start = time.time()

dw_drho = tape.gradient(w[0], rho)

time_end = time.time()
time_price_cpu = time_end - time_start
print("Runtime on CPU: ", time_price_cpu)

dw_drho

Runtime on CPU:  0.03499865531921387
